## Sprawozdanie 6 ##
### Adrian Janakiewicz ###

#### Imports ####

In [1]:
from scipy.integrate import romberg, quad, dblquad
from scipy.optimize import newton
import numpy as np
from scipy.interpolate import interp1d

<a id='zad1'></a>
***Zadanie 1.***  

In [2]:
a = 0
b = 1
f = lambda x : np.exp(-2*x)
n = 10

In [3]:
#Ad. 1 Metoda analityczna
int_f = lambda x : -0.5 * np.exp(-2*x)
I_a = int_f(1) - int_f(0)
I_a

0.43233235838169365

In [4]:
#Ad. 2 Metoda prostokątów
I_0 = (b-a)*(f((a+b)/2))
I_0

0.36787944117144233

In [5]:
#Ad. 3 Prosta metoda trapezów
I_1 = 0.5 * (b-a)*(f(a) + f(b))
I_1

0.5676676416183064

In [6]:
#Ad. 4 Prosta metoda Simpsona
I_2 = 1./6 * (b-a)*(f(a) + 4 * f((a+b)/2) + f(b))
I_2

0.4344755079870637

In [7]:
#Ad. 5 Złożona metoda trapezów
step = (b-a)/n
I_3 = sum(0.5 * step*(f(a + i * step) + f(a + (i+1) * step)) for i in range(n))    
I_3

0.4337725064184668

In [8]:
#Ad. 6 Złożona metoda Simpsona
I_4 = sum(1./6 * step*(f(a + i * step) + 4 * f((2*a+(2*i+1)*step)/2) + f(a + (i+1) * step)) for i in range(n))   
I_4

0.4323325982807031

In [9]:
#Ad. 7 Metoda Romberga
I_5 = romberg(f, a, b, tol=1e-6)
I_5

0.43233235838943557

In [10]:
# Ad.8 Metoda scipy.integrate.quad
I_6, err = quad(f, a, b)
I_6

0.4323323583816936

In [11]:
print(np.fabs(I_a-I_0))
print(np.fabs(I_a-I_1))
print(np.fabs(I_a-I_2))
print(np.fabs(I_a-I_3))
print(np.fabs(I_a-I_4))
print(np.fabs(I_a-I_5))
print(np.fabs(I_a-I_6))

0.06445291721025131
0.13533528323661276
0.0021431496053700427
0.00144014803677317
2.398990094354403e-07
7.741918217618604e-12
5.551115123125783e-17


Najbardziej zbliżone do wyniku analitycznego wyniki daje metoda quad z biblioteki scipy, jednakże medody romberga oraz złożona metoda Simpsona również dały bardzo zbliżone wyniki.

<a id='zad2'></a>
***Zadanie 2.***  

In [8]:
def ex_2_trapezoid_method():
    n = 10000
    g = lambda x: 1/np.sqrt(2*np.pi) * np.exp(-0.5*x**2)
    p0 = 0.5
    epsilon = 1e-5

    polynomial = lambda x: sum(0.5 * (x/n) *(g(i * (x/n)) + g((i+1) * (x/n))) for i in range(n)) - 0.45
    return newton(polynomial, p0, g, tol=epsilon)
    
def ex_2_simpson_method():
    n = 10000
    g = lambda x: 1/np.sqrt(2*np.pi) * np.exp(-0.5*x**2)
    p0 = 0.5
    epsilon = 1e-5
    
    polynomial = lambda x : sum(1./6 * (x/n)*(g(i * (x/n)) + 4 * g(((2*i+1)*(x/n))/2) + g((i+1) * (x/n))) for i in range(n)) - 0.45
    return newton(polynomial, p0, g, tol=epsilon)     

In [9]:
trapez = ex_2_trapezoid_method()
simps  = ex_2_simpson_method()

In [10]:
from scipy.stats import norm
# 0.45 = norm.cdf(result) - norm.cdf(0)
# 0.45 + 0.5 = norm.cdf(result)
# norm.cdf^-1 = norm.ppf 
wart_ref = norm.ppf(0.45+norm.cdf(0))
print(" Metoda  |     Wartość ref.   |   Błąd bezwzgledny ")
print("---------|--------------------|--------------------")
print("Trapezów | {} | {}". format(trapez, np.fabs(wart_ref - trapez) ))
print("Simpsona | {} | {}". format(simps , np.fabs(wart_ref - simps ) ))

 Metoda  |     Wartość ref.   |   Błąd bezwzgledny 
---------|--------------------|--------------------
Trapezów | 1.6448536306599977 | 3.7085254867008643e-09
Simpsona | 1.6448536269514749 | 2.6645352591003757e-15


<a id='zad3'></a>
***Zadanie 3.***  

In [2]:
time  = [.0, .5, 1., 2., 3., 4.]
accel = [-4.5, -4.5, -2.3, -1., -0.5, .0]
avg_speed_time = (0., 0.5)

In [4]:
acceleration = interp1d(time, accel, kind='cubic', fill_value='extrapolate')
sp = [quad(acceleration, 0, 4)[0]]
for t in time:
    sp.append(sp[0] - quad(acceleration, 0,t)[0])
sp.pop(0)
speed = interp1d(time, sp, kind='cubic', fill_value='extrapolate')

droga = np.fabs(quad(speed, 0, 4)[0])
droga2 = np.fabs(quad(speed, 0, 0.5)[0])
print("Droga hamowania {}".format(droga))
print("Średnia predkość na przedziale [0, 0.5]: {}".format(droga2/0.5))

Droga hamowania 6.331502525070518
Średnia predkość na przedziale [0, 0.5]: 5.219239267447779


<a id='zad4'></a>
***Zadanie 4.***  

In [17]:
D = 4.8
A = 0.1 
T = 0.2
f = lambda x : A * np.sin(2*np.pi*x/T)
f_prime = lambda x : 2*np.pi*A/T*np.cos(2*np.pi/T*x)
length_help = lambda x: np.sqrt(1+ f_prime(x)**2)

In [18]:
def ex_4_trapezoid_method(n):
    step = D/n
    return sum(0.5 * step*(length_help(i * step) + length_help((i+1) * step)) for i in range(n))
    
def ex_4_simpson_method(n):
    step = D/n
    return sum(1./6 * step*(length_help(i * step) + 4 * length_help(((2*i+1)*step)/2) + length_help((i+1) * step)) for i in range(n))   

def ex_4_romberg_method():
    return romberg(length_help, 0, D)

In [19]:
print("Metoda trapezów: {}".format(ex_4_trapezoid_method(1000)))
print("Metoda Simpsona: {}".format(ex_4_simpson_method(1000)))
print("Metoda Romberga: {}".format(ex_4_romberg_method()))

Metoda trapezów: 11.063484774497706
Metoda Simpsona: 11.063484774497736
Metoda Romberga: 15.825159885482952


In [20]:
import matplotlib.pyplot as plt
x = [i for i in range(1,101)]
y_t = [ex_4_trapezoid_method(i) for i in x]
y_s = [ex_4_simpson_method(i) for i in x]

plt.figure(figsize=(20, 5))
plt.scatter(x, y_t, label='Metoda trapezów')
plt.scatter(x, y_s, label='Metoda Simpsona')
plt.legend()
plt.show()

<Figure size 2000x500 with 1 Axes>

Dla badanego przypadku złożna metoda trapezów oraz złożona metoda Simpsona zwracają coraz bardziej zbliżone wyniki (zgodność nawet do 13 miejsc po przecinku).
Natomiast metoda Romberga zwraca niepoprawny wynik znacznie różny od pozostałych, aczkolwiek nie przypadkowy. Jest on bowiem zbliżony do wyników pierwszysch dwóch metod dla małej ilości przedziałów.